### Installing packages

In [15]:
!pip install pandas
!pip install numpy
!pip install openpyxl


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


### Scraping data from pdfs

Cleaning pdf

In [16]:
import pandas as pd
import openpyxl

In [19]:
f = pd.read_csv('oct22accidents.csv')

In [23]:
f.columns = f.iloc[1]

In [26]:
f.dropna(how='all', axis=1, inplace=True)

In [29]:
f.drop([0,1], inplace=True)
f.to_csv('clean.csv')